# Sythetic Images

Create images using the fitting parameters. Use list to create multiple images.

### Imports

In [ ]:
# imports
import os.path
import io_files as io
import numpy as np
import json
from fit_mir import ProgressBar
import time

### Load Setting

In [ ]:
# load default dictionary to put the values
param = {}
# set the settings file path
param["s_name"] = "settings.json"
param["s_folder"] = r"H:\github\MultipleImageRadiography"
# read the content
path_name = os.path.join(param["s_folder"], param["s_name"])
with open(path_name, "r") as f:
    param.update(json.load(f))
print(param)

### Load flats

Load data from npz files

In [ ]:
output = io.Text_Output()
Num = param["data_dirs"]

def load_result(d_path):
    result = []
    if os.path.exists(d_path):
        results = np.load(d_path, mmap_mode="r")
        out.showMessage(list(key for key in results.keys()))
        result = [results[key] for key in results.keys()]
    else:
        out.showMessage(f'File: {d_path} not found')
    return result


# set if there are two flats
param["flat2"] = True
# flat1
result1 = load_result(os.path.join(param["output_dir"], param["result_dir"], "flat_fit.npz"))
# image, adjust name of image file
# result2 = load_result(os.path.join(param["output_dir"], param["result_dir"], f"image_fit_{Num}.npz"))
# flat2 if present
result3 = []
if param["flat2"]:
    result3 = load_result(os.path.join(param["output_dir"], param["result_dir"], "flat2_fit.npz"))
    # combine results1 and results3
    result4 = list(map(lambda x, y: (x + y) / 2, result1, result3))
else:
    result4 = result1


## Create Images

Pick a sigma from the center to calculate. The flat image is used as the reference.

In [ ]:
def e_func(f):
    def g(x):
        return np.exp(f(x))

    return g

factor = param["sigma_factor"]
dtype = param["dtype"]
io.mkdir(os.path.join(param["output_dir"], param["result_dir"], f'{factor}sigma'))

start_all = time.time()
time_each = []
for N in Num:
    start_each = time.time()
    output.showMessage(f"Working on image: {N}.")
    result2 = load_result(os.path.join(param["output_dir"], param["result_dir"], f"image_fit_{N}.npz"))
    xsize, ysize = result1[0].shape
    s_image = np.zeros(result2[0].shape, dtype=float)
    PBar = ProgressBar(xsize, True)
    PBar.reset()
    for X in range(xsize):
        for Y in range(ysize):
            f_a, f_b, f_c = [result4[i][X, Y] for i in range(3, 6)]
            i_a, i_b, i_c = [result2[i][X, Y] for i in range(3, 6)]
            f_ln_fit = np.poly1d([f_c, f_b, f_a])
            i_ln_fit = np.poly1d([i_c, i_b, i_a])
            f_fit = e_func(f_ln_fit)
            i_fit = e_func(i_ln_fit)
            mu = -f_b/2*f_c
            sigma2 = -1.0 / (2.0 * f_c)
            sigma = np.sqrt(abs(sigma2))
            x_value = mu + factor*sigma
            s_image[X,Y] = i_fit(x_value)/f_fit(x_value)
        PBar.setValue(X / float(xsize) * 100.0)
    io.save_img(os.path.join(param["output_dir"], param["result_dir"], f'{factor}sigma', f'image_synth_{N}.tif'), s_image, dtype)
    PBar.setValue(100.0)
    end_each = time.time()
    time_each.append(end_each - start_each)
end_all = time.time()
each_ave = (sum(time_each) / len(time_each)) / 60.0
total = (end_all - start_all) / 60.0
output.showMessage(
        "Finished all folders.\nTotal Time = {:6.2f} min, time per folder = {:6.2f} min.".format(
            total, each_ave
        )
)